**\*\*\*\*\*\*\*Remember to restart Kerne
l for new run since the template are not refreshed\*\*\*\*\*\*\***

## Script to create a transaction bundle of multiple indiv and summary Reports


In [1]:
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
import fhirclient.r4models.bundle as B
import fhirclient.r4models.parameters as P
import fhirclient.r4models.fhirdate as D
from fhirclient.models.fhirabstractbase import FHIRValidationError 
import os, uuid
from datetime import datetime
from pprint import pprint
from pathlib import Path
#******* select for ind or summ reports ******
import ind_resource_templates_new as t 
#import summ_resource_templates as t

### Fetch MeasureReports create List 

-  *******changed to fetching mr from source using pathlib********

In [2]:
out_path = t.out_path
b_ref_list = {x for i in t.mrs for x in i } # unique set of core-references
#print(b_ref_list)
b_url_list = [f'{i.replace("/","-")}.json' for i in b_ref_list]
#print(b_url_list)
ref_map = dict(zip(b_ref_list,b_ref_list))
br_list = []
for i in b_url_list:
    path = Path() / t.base / t.source / i
    #print(path)
    #print( path.read_text() )
    br_list.append(loads(path.read_text()))
    
#source/examples/MeasureReport-summ-measurereport02.json
b_ref_list, b_url_list, ref_map, br_list, 

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/MeasureReport-measurereport03.json'

### global variables

In [ ]:
"""summ = True
use_bundle=True
single_patient = False
use_task = False
uc = ''#'mrp"""


bundle_id = (f'{"single" if t.single_patient else "multiple"}-{"summ" if t.summ else "ind"}-{t.uc}'
            f'{"" if t.summ else ("-task-" if use_task else "-obs-")}report')
print(bundle_id)

In [ ]:
bundle_type = 'transaction'
                         
'''parameter_id = '' #f'{uc}-submit-data-example'
print(parameter_id)'''

In [ ]:
'''#in_path =  "//ERICS-AIR-2/ehaas/Documents/FHIR/synthea/output/us_core_r4/fhir_10yr_hx" # local source of Synthea bundles from PC to Mac
in_path = "/Users/ehaas/Documents/Documents/FHIR/synthea/output/us_core_r4/fhir_10yr_hx"
#out_path = '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/'  # append forward slash
out_path ='r4'  #local output dir
#out_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/source/examples"  # DEqm source'''

t.out_path

In [ ]:
'''
if single_patient:
    ref_map = dict(zip(t.mrp1_core_references,t.mrp1_core_references)) # init the ref with old:new=old
    my_resources = t.mrp1  # get all core resource templates from use case list

else: # mult patients or summ
    mult_ref_mrp = (i['core_references'] for i in t.mrs)
    # mult_ref_mrp =  set(t.mrp1_core_references + t.mrp2_core_references + t.mrp3_core_references)
    #ref_map = dict(zip(mult_ref_mrp,mult_ref_mrp))

    #mult_mrp = t.mrp1 + t.mrp2 + t.mrp3  # take three list and combine
    my_resources = [i['id'] for i in t.mrs]
    
mult_ref_mrp, my_resources'''

## Some Functions

###  write to file

In [ ]:
def write_file(r): # write file
    with open(f'{out_path}/Bundle-{bundle_id}.json', 'w', encoding='utf-8') as f:
        f.write(r)

### validate

In [ ]:
# *********************** validate Resource ********************************

def validate(r):

    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-1'
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/{r.resource_type}/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### parse out patient url from Synthea bundle object


In [ ]:
def get_resource_url(b, type):
    my_url = [e.fullUrl for e in b.entry if e.resource.resource_type ==type]
    return(my_url[0]) 

### get resource from Synthea bundle object by id

-  return as dict
-  make generic later

In [ ]:
def get_resource_by_url(b,my_url):
    '''my_encounters = [e.resource for e in b.entry
        if e.resource.resource_type == 'Encounter']
    for my_encounter in my_encounters:
        print( my_encounter.id, my_encounter.type[0].coding[0].as_json() )
        #return(my_encounter[0].as_json())# assume only one / Bundle'''
    my_resource = [e.resource for e in b.entry
        if e.fullUrl == my_url]
    print(my_url, my_resource)
    return(my_resource[0].as_json())# assume only one / Bundle
 

### update my_resources list and reference map


In [ ]:
def add_resource_by_url(my_patient_data,my_url,r_type,num=0):
    ref_map[f'{r_type}/{r_type.lower()}0{num+1}'] = my_url
    my_resource = get_resource_by_url(my_patient_data,my_url)
    # if organization make sure has an NPI
    if my_resource["resourceType"] == "Organization":
        my_resource["identifier"] = t.org_npi
    # if encoounter make an inpatient encounter
    elif my_resource["resourceType"] == "Encounter":
        my_resource["class"]['code'] == "AMB"
        my_resource["type"] = t.enc_type
        # use the same organization for all the reports in the bundle
        if num != 0:
            my_resource["serviceProvider"] = {"reference":ref_map["Organization/organization01"]}
            my_resource["location"][0]["location"] = {"reference":ref_map["Location/location01"]}
    elif my_resource["resourceType"] == "Patient": 
        my_resource.pop('deceasedDateTime', None) # make sure is alive...
        # remove illegal extensions
        my_resource['extension'].pop()
        my_resource['extension'].pop() 
    my_resources.append(my_resource)
return

### Get files in current path

- get only 1
- convert the files to FHIR Bundle object


In [ ]:
def get_synth_patient_data(num=1):
    pyfhir = []
    my_patient_files = [x for i,x in enumerate(os.listdir(in_path)) if i < num]
    pprint(my_patient_files)
    for my_patient_file in my_patient_files:
        with open(f'{in_path}/{my_patient_file}', encoding='utf-8') as f:
            r = f.read()
            pyfhir.append(B.Bundle(loads(r)))
    return(pyfhir)

## Main part of Code

### 1. create Bundle 'b'  change the id for unique Bundles!!!

In [ ]:
b = B.Bundle()
b.id =  bundle_id
b.type = bundle_type
b.timestamp = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
b.as_json()

### 1. create Parameters resource 'p'  change the id for unique Bundles!!!

In [ ]:
p = P.Parameters()
p.id =  bundle_id
p.as_json()

### 2.  Add Core resources to resource bundle list

- get all core resource templates from use case list

### 3.  Add Patient resources to resource bundle list

- first get synthea patient bundle for 1 patient
- parse out patient resource
- add to list as patient01
- make alive if deceased

### 3.  Add Encounter resources to resource bundle list

- use synthea patient bundle for same patient as above
- parse out encounter resources and look at type
- choose id for inpatient encounter
- use id to:
    - add to list as encounter
    - replace all references id
    
### based upon the Encounter add in the organization01, Location and patient01

- use the uuid to add in these resources
- update the ref_map

### Create a mapping of ids to uuids

- get id from resource
- map to uuid (create a dict of {urn:old_ref})
- remove id element
- replace all old refs in bundle with new urns


In [ ]:
first = True
b.entry = []
'''
num = 1 if single_patient else 3

# get patient data as fhir object
my_patient_data_list = get_synth_patient_data(num)


for i,my_patient_data in enumerate(my_patient_data_list):
    # get patient0n as dict
    r_type = 'Patient'
    print(f'i={i}')
    
    
    my_url = get_resource_url(my_patient_data, r_type)
    add_resource_by_url(my_patient_data,my_url,r_type,i)

    # get encounter as dict
    r_type = 'Encounter'
    my_url = get_resource_url(my_patient_data, r_type)
    add_resource_by_url(my_patient_data,my_url,r_type,i)

    my_encounter = my_resources[-1]
    # get practitioner01 from encounter
    my_url = my_encounter['participant'][0]['individual']['reference']
    r_type = 'Practitioner'
    add_resource_by_url(my_patient_data,my_url,r_type,i)

    if i == 0:
        # get organization01 from encounter
        my_url = my_encounter['serviceProvider']['reference']
        r_type = 'Organization'
        add_resource_by_url(my_patient_data,my_url,r_type)
        # get location01 from encounter
        my_url = my_encounter['location'][0]['location']['reference']
        r_type = 'Location'
        add_resource_by_url(my_patient_data,my_url,r_type)
'''   
#if use_bundle:


'''
variables=
b_id_list, 
b_mrref_list, 
b_ref_list, 
b_ref_list, 
b_url_list, 
ref_map, 
br_list, 
'''

for resource in br_list:  # look at each dict in list and bundle
    resource_type= resource["resourceType"]
    
    # make sure that evaluatedResource is aligned
    if resource_type == 'MeasureReport' and not t.use_task:
        try:
            er = resource['evaluatedResource'][0]['reference']
        except KeyError:
            pass
        else:
            er = er.replace('Task', 'Observation')
            er = er.replace('task', 'observation')
            resource['evaluatedResource'][0]['reference'] = er


    # remove text from resource
    resource.pop('text', None)       
    # remove id from resource if not a uuid and replace with uuid
    #print(template["id"])
    old_id =  resource.pop('id',None)
    try: # check if a valid UUID
        val = uuid.UUID(old_id, version=4)
        new_urn = f'urn:uuid:{old_id}'
    except ValueError:
        # If it's a value error, then the string 
        # is not a valid hex code for a UUID.
        # print('invalid')
        new_urn = uuid.uuid1().urn # new urn for resource
        ref_map[f'{resource_type}/{old_id}'] = new_urn #update the maps to uuid
    #print(ref_map , '\n')
    e = B.BundleEntry(dict
        (
        resource = resource,
        fullUrl = new_urn,
        )
      )
    if bundle_type in ['transaction', 'batch']:
        e.request = B.BundleEntryRequest(dict
                    (
                    method = 'POST',
                    url =  resource['resourceType']
                    )
                    )

    if( (resource_type != 'Observation' and t.use_task)
        or (resource_type != 'Task' and not t.use_task) ): 
        b.entry.append(e)
pprint(ref_map)
b_json = dumps(b.as_json(),indent =3)
# print(b_json)
for old_ref, new_ref in ref_map.items():
    b_json = b_json.replace(old_ref,new_ref)
"""for bad, good in t.qa_issues.items():
    b_json = b_json.replace(bad,good)"""
print(b_json)
print('.......validating')
r = validate(b)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
write_file(b_json)
                 
'''else:  # use parameters
    p = P.Parameters()
    p.id =  bundle_id
    pprint(ref_map)             

    for resource in my_resources:  # all templates as dict in mrp
        resource_type= resource["resourceType"]
'''
        
        
              